In [41]:
import time
from itertools import chain
import email
import imaplib
import re


imap_ssl_host = ''  # imap.mail.yahoo.com
imap_ssl_port = 993
username = ''
password = ''


list_response_pattern = re.compile(r'\((?P<flags>.*?)\) "(?P<delimiter>.*)" (?P<name>.*)')

def parse_list_response(line):
    flags, delimiter, mailbox_name = list_response_pattern.match(line.decode()).groups()
    mailbox_name = mailbox_name.strip('"')
    return (flags, delimiter, mailbox_name)


def getMailboxes(server):
    mailboxes = []
    status, resp = server.list()
    for line in resp:
        flags, delimiter, mailbox_name = parse_list_response(line)
        mailboxes.append(mailbox_name)
        # print 'Parsed response:', (flags, delimiter, mailbox_name)

    print(mailboxes)
    # print("Mailboxes:")
    # for box in mailboxes:
    #     print("->",box)
    # print("\n")



with imaplib.IMAP4_SSL(imap_ssl_host, imap_ssl_port) as server:
    server.login(username, password)
    getMailboxes(server)
    
    
    typ, data = server.select('AWS')
    print(typ, data)


    # print(server.status("AWS", names=""))

    typ, data = server.search(None, 'UNSEEN')
    for x in data[0].split():
        # print(x)
        typ, data = server.fetch(x, "ENVELOPE")
            #server.fetch(x, '(RFC822)')
        print('Message %s\n%s\n' % (x, data.decode().split(" ")))


    # result, data = server.uid('search', None, search_string(uid_max, criteria))

    # uids = [int(s) for s in data[0].split()]
    # if uids:
    #     uid_max = max(uids)
    #     # Initialize `uid_max`. Any UID less than or equal to `uid_max` will be ignored subsequently.

    server.close()
    server.logout()


ConnectionRefusedError: [WinError 10061] Es konnte keine Verbindung hergestellt werden, da der Zielcomputer die Verbindung verweigerte